## HyperParameter tuning
### Efficient Net B16 - Food101 dataset
- number of parameters : 9,109,994

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 13.5 MB/s eta 0:00:00


In [1]:
# Checking versions of libraries for google colab: torch, torchvision

try:
    import torch
    import torchvision
    print(f'torch version: {torch.__version__}')
    print(f'torchvision version: {torchvision.__version__}')
    assert int(torch.__version__.split('.')[1]) >= 12, 'torch version should be 1.12+'
    assert int(torchvision.__version__.split('.')[1]) >= 13, 'torchvision version should be 0.13+'
except:
    print(f'[INFO] torch/torchvision versions not as required, installing required versions.')


c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch version: 1.13.1
torchvision version: 0.14.1


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

from torchmetrics import Accuracy

from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter

import sys
sys.path.append('..')
sys.path.append('./drive/MyDrive/Colab Notebooks/Paper Replicating/')
from python_scripts import engine, data_setup, utils

In [3]:
CUR_FORDER = './drive/MyDrive/Colab Notebooks/Paper Replicating/'
BATCH_SIZE = 32


In [4]:
# Checking GPU

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [5]:
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2_food101_transforms = effnetb2_weights.transforms()

In [6]:
data_dir = Path('../data/extracted')

train_data_effnetb2_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=effnetb2_food101_transforms,
    download=True
)
test_data_effnetb2_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=effnetb2_food101_transforms,
    download=True
)

class_names_food101 = train_data_effnetb2_food101.classes

train_dataloader_effnetb2_food101 = DataLoader(
    dataset=train_data_effnetb2_food101,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dataloader_effnetb2_food101 = DataLoader(
    dataset=test_data_effnetb2_food101,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [7]:
print(train_data_effnetb2_food101)
print(train_dataloader_effnetb2_food101)
print(len(class_names_food101))

Dataset Food101
    Number of datapoints: 75750
    Root location: ..\data\extracted
    split=train
    StandardTransform
Transform: ImageClassification(
               crop_size=[288]
               resize_size=[288]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BICUBIC
           )
101


In [8]:
dropout_p_list = [0]
learning_rate_list = [1e-4]
weight_decay_list = [0, 0.01]
epochs_list = [20]
batch_size_list = [32]

In [9]:
sub32_train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_effnetb2_food101,
    split_size=0.3,
    seed=42
)
sub32_test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_effnetb2_food101,
    split_size=0.3,
    seed=42
)

tuning_results = engine.effnetb2_HP_tune_train_TSboard(
    model_weights=effnetb2_weights,
    train_dataset=sub32_train_dataset,
    test_dataset=sub32_test_dataset,
    dropout_p_list=dropout_p_list,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_writer=True,
    device=device
)

[INFO] Splitting dataset of length 75750 into splits of size: 22725 and 53025
[INFO] Splitting dataset of length 25250 into splits of size: 7575 and 17675
[INFO] Creating SummaryWriter saving to ../runs\2023-02-28\230227_effnetb2_food101_test\EffNet_b2_food101\DP_0_LR_0.0001_WD_0.01_EP_5_BS_32


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 4.2882, Train_acc: 0.1837 | Test_loss: 3.9746, Test_acc: 0.3658


 20%|██        | 1/5 [06:49<27:18, 409.52s/it]

Epoch: 1 | Train_loss: 3.7476, Train_acc: 0.3894 | Test_loss: 3.5195, Test_acc: 0.4478


...

 60%|██████    | 3/5 [20:41<13:49, 414.91s/it]

Epoch: 3 | Train_loss: 3.7565, Train_acc: 0.4291 | Test_loss: 3.7187, Test_acc: 0.4502


 80%|████████  | 4/5 [27:32<06:53, 413.27s/it]

Epoch: 4 | Train_loss: 3.7157, Train_acc: 0.4326 | Test_loss: 3.7036, Test_acc: 0.4441


100%|██████████| 5/5 [34:15<00:00, 411.10s/it]


In [10]:
sub32_train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_effnetb2_food101,
    split_size=1,
    seed=42
)
sub32_test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_effnetb2_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.effnetb2_HP_tune_train_TSboard(
    model_weights=effnetb2_weights,
    train_dataset=sub32_train_dataset,
    test_dataset=sub32_test_dataset,
    dropout_p_list=dropout_p_list,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_writer=True,
    device=device
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-02-28\230227_effnetb2_food101_test\EffNet_b2_food101\DP_0_LR_0.0001_WD_0.01_EP_5_BS_32


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 3.7815, Train_acc: 0.3333 | Test_loss: 3.1546, Test_acc: 0.4830


 20%|██        | 1/5 [26:23<1:45:35, 1583.91s/it]

Epoch: 1 | Train_loss: 2.9992, Train_acc: 0.4620 | Test_loss: 2.7179, Test_acc: 0.5249


...

 60%|██████    | 3/5 [1:08:48<45:52, 1376.36s/it]

Epoch: 3 | Train_loss: 3.6908, Train_acc: 0.4202 | Test_loss: 3.6135, Test_acc: 0.4703


 80%|████████  | 4/5 [1:31:45<22:56, 1376.41s/it]

Epoch: 4 | Train_loss: 3.6888, Train_acc: 0.4173 | Test_loss: 3.6268, Test_acc: 0.4788


100%|██████████| 5/5 [1:54:41<00:00, 1376.26s/it]


In [13]:
import pandas as pd
pd.DataFrame(tuning_results)

,dropout_p,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0.0,0.0001,0.05,5,32,4.411838,4.625461,0.09280303,0.020833334
1,0.0,0.0001,0.10,5,32,4.4178963,4.6091146,0.09280303,0.010416667
2,0.0,0.0001,0.30,5,32,4.541189,4.601899,0.03125,0.03125
3,0.0,0.0010,0.05,5,32,2.94697,4.6488004,0.87689394,0.041666668
4,0.0,0.0010,0.10,5,32,3.1120822,4.5333424,0.9592803,0.0625
5,0.0,0.0010,0.30,5,32,3.5981262,4.544896,0.80492425,0.020833334
6,0.0,0.0100,0.05,5,32,0.2725514,4.325723,1.0,0.1344697
7,0.0,0.0100,0.10,5,32,0.7193756,4.4286203,1.0,0.083333336
8,0.0,0.0100,0.30,5,32,1.8089956,4.4606037,0.88825756,0.0625
9,0.3,0.0001,0.05,5,32,4.437482,4.623685,0.072916664,0.010416667
